In [101]:
import os
import pandas as pd
#import nltk
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import os
import re
import numpy as np
#os.getcwd()

In [159]:
path = 'C:/Users/axmarr/Documents/Varios/maestria/NLP/NLP-master/'

#path = 'C:/Users/Kikes/Documents/GitHub/NLP/df_comms.csv'

comentarios = pd.read_csv(path+'df_comms.csv', encoding='utf-8', index_col=0)

In [160]:
df = comentarios.copy().head(10)

In [161]:
df['comm_msg'].replace('', np.nan, inplace = True)

In [162]:
df['emojis'].replace('', np.nan, inplace = True)

In [163]:
df['solo_emoji'] = np.where(df.comm_msg.isnull()&df.emojis.notnull(),1,0)
#comentarios[comentarios.solo_emoji == 1]

In [164]:
#dropeo filas con comentarios y sin emojis
df.dropna( subset = ['comm_msg', 'solo_emoji'], inplace = True, how = 'all' )

In [166]:
#Desde acá: https://nlp.stanford.edu/software/CRF-NER.shtml#Download
#Se baja: https://nlp.stanford.edu/software/stanford-ner-2017-06-09.zip
#Se descomprime el .zip y a la carpeta que está adentro se le cambia el nombre de: stanford-ner-2017-06-09 a: stanford-ner 

#import os
java_path = "C:/Program Files/Java/jre1.8.0_131/bin/java.exe"
#java_path = 'C:/Program Files (x86)/Java/jre1.8.0_131/bin/java.exe'

os.environ['JAVAHOME'] = java_path

#os.getcwd()
#Hay que cambiar el directorio y que apunte donde está la carpeta que descomprimimos
os.chdir("C:/Users/axmarr/Documents/Varios/maestria/NLP/")
#os.chdir('C:/Users/Kikes/Documents/maestria/NLP/')

# -*- coding: utf-8 -*-

#from nltk.tag import StanfordNERTagger
#from nltk.tokenize import word_tokenize

st = StanfordNERTagger('stanford-ner/classifiers/spanish.ancora.distsim.s512.crf.ser.gz',
					   'stanford-ner/stanford-ner.jar',
					   encoding='utf-8')

C:\Users\axmarr\Documents\Python\WinPython-64bit-3.6.1.0Qt5\python-3.6.1.amd64\lib\site-packages\nltk\tag\stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordNERTagger, self).__init__(*args, **kwargs)


In [167]:
def extraeNombres(comm):
    comm_str = str(comm)#comm.to_json()
    #comm_token = word_tokenize(comm_str)
    comm_token = word_tokenize(comm_str)
    #comm_token = word_tokenize(comm)
    comm_class = st.tag(comm_token)
    comm_list_per = [pers for pers in comm_class if pers[1] =='PERS']
    comm_per = re.sub(r'[\[\]\(\)\'\,]', "", str([i[0] for i in comm_list_per]))
    return comm_per

In [172]:
#Ver como optimizar este punto, el cuello de botella esta en st.tag()
#https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6

#pd.options.mode.chained_assignment = None
df['nombres'] = df['comm_msg'].apply(lambda x: extraeNombres(x))
df['nombres'].replace( '', np.nan, inplace = True )
#comentarios['nombres'] = comentarios['comm_msg'].apply(lambda x: extraeNombres(x))
#pd.options.mode.chained_assignment = 'warn'

In [173]:
n_comm = len(df)

for c in range(n_comm):
    ext_name = str(df.nombres[c])
    ext_comm = str(df.comm_msg[c])
    comm_no_name = ext_comm.replace(ext_name,"")
    df.comm_msg[c] = comm_no_name

C:\Users\axmarr\Documents\Python\WinPython-64bit-3.6.1.0Qt5\python-3.6.1.amd64\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [174]:
df['comm_msg'].replace( '', np.nan, inplace = True )

In [175]:
df['solo_nombre'] = np.where( df.comm_msg.isnull()&df.nombres.isnull()&df.emojis.notnull(), 1, 0 )

In [176]:
df['tiene_nombre'] = np.where( df.nombres.notnull(), 1, 0 )

In [177]:
df['tiene_emoji'] = np.where( df.emojis.notnull(), 1, 0 )

In [178]:
df['largo_comm'] = df['comm_msg'].apply(lambda x: len(x.replace(" ","")))

In [179]:
df['comm_msg'] = df['comm_msg'].apply(lambda x: re.sub(r'[!#$%&\'()*+,-./:;<=>?@\[\]^_`{|}~¡]', " ", x))

In [180]:
#https://stackoverflow.com/questions/19130512/stopword-removal-with-nltk
#from nltk.corpus import stopwords
stop = set(stopwords.words('spanish'))
sw2 = set(line.strip() for line in open('stop_words_acentos.txt'))
stop2 = stop|sw2

In [181]:
#comm  = df['comm_msg'][1]

def quitaStopWords(comm):

    split_comm = comm.lower().split()
    comm_no_stop = [i for i in split_comm if i not in stop2]
    comm_str = ' '.join(comm_no_stop)
    comm_str_no_short = ' '.join(word for word in comm_str.split() if len(word)>2)
    return comm_str_no_short

In [182]:
df['comm_msg_clean'] = df['comm_msg'].apply( lambda x: quitaStopWords( x ) )

In [183]:
df['largo_comm_clean'] = df['comm_msg_clean'].apply(lambda x: len(x.replace(" ","")))

In [184]:
df['ratio_largo_clean'] = df['largo_comm_clean']/df['largo_comm']

In [186]:
df.to_csv( 'df_final.csv', encoding = 'utf-8')

In [122]:
#df[['comm_msg_clean', 'comm_msg', 'largo_comm', 'largo_comm_clean', 'ratio_largo_clean']]

In [123]:
#Nube de tags
#https://www.kaggle.com/ngyptr/python-nltk-sentiment-analysis

In [124]:
#os.chdir('C:/Users/Kikes/Documents/maestria/NLP/')
#df_com = pd.read_csv('comentarios.csv', encoding='utf-8', index_col = 0)
#df_com.head(10)